In [4]:
# composite image generation by copy-and-paste followed by image blending.
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid, draw_bbox_on_image
import cv2
test_dir = '../source/'
img_list = ['21.jpeg']
bbox_list = [[1000, 895, 1480, 1355], [1170, 944, 2331, 3069]] 
for i,img_name in enumerate(img_list):
    bg_img  = test_dir + 'background/' + img_name
    bbox    = bbox_list[i] # x1,y1,x2,y2
    fg_img  = test_dir + 'foreground/' + img_name
    fg_mask = test_dir + 'foreground_mask/' + img_name.replace('.jpeg', '.png')
    # generate composite images by naive methods
    comp_img1, comp_mask1 = get_composite_image(fg_img, fg_mask, bg_img, bbox, 'none')
    comp_img2, comp_mask2 = get_composite_image(fg_img, fg_mask, bg_img, bbox, 'gaussian')
    comp_img3, comp_mask3 = get_composite_image(fg_img, fg_mask, bg_img, bbox, 'poisson')
    vis_list = [bg_img, fg_img, comp_img1, comp_mask1, comp_img2, comp_mask2, comp_img3, comp_mask3]
    # visualization results
    grid_img  = make_image_grid(vis_list, cols=4)
    cv2.imwrite(f'../docs/_static/image/generatecomposite_result{i+1}.jpg', grid_img)

ValueError: could not broadcast input array from shape (460,480) into shape (129,24)

In [ ]:
# color-transfer: transfer foreground color to fit background scene.
from libcom import color_transfer
from libcom.utils.process_image import make_image_grid
import cv2
comp_img1  = '../tests/source/composite/1.jpg'
comp_mask1 = '../tests/source/composite_mask/1.png'
trans_img1 = color_transfer(comp_img1, comp_mask1)
comp_img2  = '../tests/source/composite/8.jpg'
comp_mask2 = '../tests/source/composite_mask/8.png'
trans_img2 = color_transfer(comp_img2, comp_mask2)
# visualization results
grid_img  = make_image_grid([comp_img1, comp_mask1, trans_img1, 
                            comp_img2, comp_mask2, trans_img2], cols=3)
cv2.imwrite('../docs/_static/image/colortransfer_result1.jpg', grid_img)

In [ ]:
# predict harmony score for a composite image.
from libcom import HarmonyScoreModel
from libcom.utils.process_image import make_image_grid
import cv2
net = HarmonyScoreModel(device=0, model_type='BargainNet')
test_dir   = '../tests/harmony_score_prediction/'
img_names  = ['vaulted-cellar-247391_inharm.jpg', 'ameland-5651866_harm.jpg']
vis_list,scores = [], []
for img_name in img_names:
    comp_img  = test_dir + 'composite/' + img_name
    comp_mask = test_dir + 'composite_mask/' + img_name
    score     = net(comp_img, comp_mask)
    vis_list += [comp_img, comp_mask]
    scores.append(score)
grid_img  = make_image_grid(vis_list, text_list=[f'harmony_score:{scores[0]:.2f}', 'composite-mask', f'harmony_score:{scores[1]:.2f}', 'composite-mask'])
cv2.imwrite('../docs/_static/image/harmonyscore_result1.jpg', grid_img)

In [ ]:
# Object Placement Assessment (OPA) score prediction
from libcom import OPAScoreModel
from libcom import get_composite_image
from libcom.utils.process_image import make_image_grid
import cv2
net = OPAScoreModel(device=0, model_type='SimOPA')
test_dir  = './source'
bg_img    = 'source/background/17.jpg'
fg_img    = 'source/foreground/17.jpg'
fg_mask   = 'source/foreground_mask/17.png'
bbox_list = [[475, 697, 1275, 1401], [475, 300, 1275, 1004]]
comp1, comp_mask1 = get_composite_image(fg_img, fg_mask, bg_img, bbox_list[0])
comp2, comp_mask2 = get_composite_image(fg_img, fg_mask, bg_img, bbox_list[1])
score1 = net(comp1, comp_mask1)
score2 = net(comp2, comp_mask2)
grid_img  = make_image_grid([comp1, comp_mask1, comp2, comp_mask2], text_list=[f'opa_score:{score1:.2f}', 'composite-mask', f'opa_score:{score2:.2f}', 'composite-mask'])
cv2.imwrite('../docs/_static/image/opascore_result1.jpg', grid_img)

In [ ]:
# Foreground Object Search (FOS) score prediction
from libcom.utils.process_image import make_image_grid
from libcom import FOSScoreModel
import cv2
import torch

task_name = 'fos_score_prediction'
MODEL_TYPE = 'FOS_D' # choose from 'FOS_D', 'FOS_E'
background = '../tests/source/background/f80eda2459853824_m09g1w_b2413ec8_11.png'
fg_bbox    = [175, 82, 309, 310] # x1,y1,x2,y2
foreground = '../tests/source/foreground/f80eda2459853824_m09g1w_b2413ec8_11.png'
foreground_mask = '../tests/source/foreground_mask/f80eda2459853824_m09g1w_b2413ec8_11.png'
net = FOSScoreModel(device=0, model_type=MODEL_TYPE)
score = net(background_image, foreground_image, bounding_box, foreground_mask=foreground_mask)
grid_img  = make_image_grid([background_image, foreground_image, composite_image], text_list=[f'fos_score:{score:.2f}'])
cv2.imshow('fos_score_demo', grid_img)


In [ ]:
# controllable image composition
from libcom import ControlComModel
from libcom.utils.process_image import make_image_grid, draw_bbox_on_image
import cv2
img_names = ['6c5601278dcb5e6d_m09728_f5cd2891_17.png', '000000460450.png']
bboxes    = [[130, 91, 392, 271], [134, 158, 399, 511]] # x1,y1,x2,y2
test_dir  = '../tests/controllable_composition/'
for i in range(len(img_names)):
    bg_img  = test_dir + 'background/' + img_names[i]
    fg_img  = test_dir + 'foreground/' + img_names[i]
    bbox    = bboxes[i]
    mask    = test_dir + 'foreground_mask/' + img_names[i]
    net     = ControlComModel(device=0)
    comp    = net(bg_img, fg_img, bbox, mask, task=['blending', 'harmonization'])
    bg_img  = draw_bbox_on_image(bg_img, bbox)
    grid_img = make_image_grid([bg_img, fg_img, comp[0], comp[1]])
    cv2.imwrite('../docs/_static/image/controlcom_result{}.jpg'.format(i+1), grid_img)
